# Crashes Near Point

Find the crash incident_id for a specific crash given its location and time. 

In [1]:
import folium
import datetime
import pandas as pd

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
# df = cda.crash_dataframe(verbose=True)

In [3]:
# University Blvd and E 3rd Ave (2022,12,14) incident_id = 2022639830
# lat_lon = [39.720934, -104.959436]

# Speer Blvd and W 11th Ave
# lat_lon = [39.73378086237446, -104.99382870794332]

# Sheridan and W 38th Ave
lat_lon = [39.76932995840248, -105.05322879106377]

# 18th St and Wazee - 2023-01-07
# lat_lon = [39.753095972145644, -104.99765515593928]

# 17th Ave and Humboldt
# lat_lon = [39.74325582993406, -104.96969463423684]

# Evans and Holly
# lat_lon = [39.678456222120396, -104.92219658639499]

In [4]:
# crash occurred within 0.05 mile distance of the given latitude and longitude

query = f"""
select
*
, ST_Distance(
    geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)
) as distance_feet

from crashes

where geo_lon is not null
and ST_Distance(geom_denver, ST_Transform(ST_SetSRID(ST_MakePoint({lat_lon[1]},{lat_lon[0]}), 4326), 3502)) < 264

order by reported_date
"""
# and extract(year from reported_date) >= 2022

nearby = pd.read_sql(query, cda.conn)

In [5]:
display_cols = [
    'crash_date'
    , 'top_traffic_accident_offense'
    , 'incident_address'
    , 'bicycle_ind'
    , 'pedestrian_ind'
    , 'sbi'
    , 'fatality'
]
# nearby.to_clipboard()
# nearby[display_cols]

In [6]:
nearby.groupby(nearby.reported_date.dt.year).agg(
    crashes=('incident_id', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date,,,,,
2013,11,0,0,1,0
2014,16,0,0,0,0
2015,12,0,0,1,0
2016,15,0,0,0,0
2017,13,0,0,0,0
2018,18,0,0,1,0
2019,14,0,0,0,0
2020,3,0,0,0,0
2021,6,0,1,1,0


In [7]:
nearby[display_cols].tail(5)

,crash_date,top_traffic_accident_offense,incident_address,bicycle_ind,pedestrian_ind,sbi,fatality
109,2022-04-09,TRAF - ACCIDENT - HIT & RUN,W 38TH AVE / N SHERIDAN BLVD,0.0,0.0,False,False
110,2022-06-29,TRAF - ACCIDENT,N SHERIDAN BLVD / W 38TH 39TH ALY,0.0,0.0,False,False
111,2022-10-04,TRAF - ACCIDENT - HIT & RUN,3800 BLOCK N SHERIDAN BLVD,0.0,0.0,False,False
112,2022-11-12,TRAF - ACCIDENT - DUI/DUID,W 38TH AVE / N SHERIDAN BLVD,0.0,0.0,False,False
113,2022-12-09,TRAF - ACCIDENT - FATAL,N SHERIDAN BLVD / W 38TH AVE,1.0,0.0,False,True


In [8]:
cda.conn.close()